In [10]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

# Preprocessing the dataset THIS IS THE OLD ONE WORK ON THE OTHER ONE

This part of the code is essentially preparing the data for a machine learning model, transforming the text data into numerical form, and splitting the data into training and testing sets.

The combined dataset is loaded from a CSV file using pandas' read_csv function and all news headlines for each record (each day) are concatenated into a single string. A CountVectorizer is initialized to convert the headlines into a matrix of token counts. The maximum number of features is set to 5000, but this can be adjusted based on computational capacity. A LabelEncoder is used to prepare the output matrix (Y) by transforming the labels into normalized encoding.

The dataset is split into training and testing sets based on specific date ranges. The variables X_train, X_test, Y_train, and Y_test are defined in a later cell, which split the X and Y matrices into training and testing sets based on the indices of the original train and test dataframes.

In [11]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

# Load the dataset
data = pd.read_csv('Datasets/Combined_News_DJIA.csv')

# Concatenate all news headlines into a single string for each record more efficiently
data['All_Headlines'] = data.iloc[:, 2:].fillna('').agg(' '.join, axis=1)

# Initialize a CountVectorizer with efficient memory usage
vectorizer = CountVectorizer(max_features=5000, dtype='uint8')  # Using uint8 for memory efficiency
X = vectorizer.fit_transform(data['All_Headlines'])

# Display the shape of X
print("Shape of X:", X.shape)

# Prepare the output matrix with LabelEncoder
Y = LabelEncoder().fit_transform(data['Label'])

# Display the first few processed records to check everything went as expected
print(data[['Date', 'All_Headlines', 'Label']].head())

# Split the dataset into training and testing sets based on the provided date ranges
train = data[(data['Date'] >= '2008-08-08') & (data['Date'] <= '2014-12-31')]
test = data[(data['Date'] >= '2015-01-02') & (data['Date'] <= '2016-07-01')]

# Print the shape of train and test data
print("Shape of train data:", train.shape)
print("Shape of test data:", test.shape)


Shape of X: (1989, 5000)
         Date                                      All_Headlines  Label
0  2008-08-08  b"Georgia 'downs two Russian warplanes' as cou...      0
1  2008-08-11  b'Why wont America and Nato help us? If they w...      1
2  2008-08-12  b'Remember that adorable 9-year-old who sang a...      0
3  2008-08-13  b' U.S. refuses Israel weapons to attack Iran:...      0
4  2008-08-14  b'All the experts admit that we should legalis...      1
Shape of train data: (1611, 28)
Shape of test data: (378, 28)


In [12]:
X_train, X_test = X[data.index.isin(train.index)], X[data.index.isin(test.index)]
Y_train, Y_test = Y[data.index.isin(train.index)], Y[data.index.isin(test.index)]

In [13]:
# Convert sparse matrix to dense
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit on the training set
scaler.fit(X_train_dense)

# Transform both the training set and the test set
X_train_std = scaler.transform(X_train_dense)
X_test_std = scaler.transform(X_test_dense)


In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder, MaxAbsScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Load and preprocess dataset
data = pd.read_csv('Datasets/Combined_News_DJIA.csv')
data['All_Headlines'] = data.iloc[:, 2:].fillna('').apply(lambda x: ' '.join(x), axis=1)

vectorizer = CountVectorizer(max_features=5000, stop_words='english')
X = vectorizer.fit_transform(data['All_Headlines']).toarray()
Y = LabelEncoder().fit_transform(data['Label'])

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

scaler = MaxAbsScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Building the baseline model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_scaled, Y_train, validation_data=(X_test_scaled, Y_test), epochs=10, batch_size=256)

# Evaluation
evaluation = model.evaluate(X_test_scaled, Y_test)
print(f'Test Loss: {evaluation[0]}, Test Accuracy: {evaluation[1]}')

# Enhancing the model - Adding more layers and neurons
enhanced_model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

enhanced_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the enhanced model
enhanced_history = enhanced_model.fit(X_train_scaled, Y_train, validation_data=(X_test_scaled, Y_test), epochs=20, batch_size=128)

# Evaluation of the enhanced model
enhanced_evaluation = enhanced_model.evaluate(X_test_scaled, Y_test)
print(f'Enhanced Model - Test Loss: {enhanced_evaluation[0]}, Test Accuracy: {enhanced_evaluation[1]}')


Epoch 1/10


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.4940 - loss: 0.6970 - val_accuracy: 0.5754 - val_loss: 0.6867
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6393 - loss: 0.6663 - val_accuracy: 0.5578 - val_loss: 0.6861
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7001 - loss: 0.6340 - val_accuracy: 0.4899 - val_loss: 0.6945
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7669 - loss: 0.5860 - val_accuracy: 0.4774 - val_loss: 0.7028
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8042 - loss: 0.5206 - val_accuracy: 0.4724 - val_loss: 0.7214
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8662 - loss: 0.4476 - val_accuracy: 0.4648 - val_loss: 0.7533
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9065 - loss: 0.3460 - val_accuracy: 0.4673 - val_loss: 0.8303
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9522 - loss: 0.2567 - val_accuracy: 0.4774 - val_loss: 0.8939
Epoch 9/10
7/7 ━━━